In [1]:
import os
#transformers version 4.22.0, cuda 11.4, torch 1.12+cu133
import malaya
import torch
import pandas as pd
pd.set_option('display.max_colwidth', None)

2023-08-08 10:23:50.303385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 10:23:50.371565: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-08 10:23:50.812293: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2023-08-08 10:23:50.812343: W tensorflow/compiler/xla/stream

In [2]:
train = pd.read_json(r"/home/aisyahrzak/question-generation/data/train.jsonl", lines=True)
dev = pd.read_json(r"/home/aisyahrzak/question-generation/data/dev.jsonl", lines=True)

test = pd.read_json("sukanz.jsonl",lines=True)

In [3]:
train.shape

(37931, 3)

In [4]:
dev.shape

(3271, 3)

### View Dataset

In [5]:
train.tail()

,source_text,target_text,task
37926,"generate questions: Isaac Newton (1643-1727) mewarisi konsepsi mekanikal Descartes tentang jirim. Dalam ketiga ""Rules of Reasoning in Philosophy"" beliau, Newton menyenaraikan sifat-sifat sejagat jirim sebagai ""sambungan, kekerasan, kebolehpercayaan, mobiliti, dan inersia"". Begitu juga dalam Optik dia menyangkal bahawa Tuhan mencipta jirim sebagai ""zarah pepejal, besar, keras, tidak dapat ditembusi, boleh bergerak"", yang ""... walaupun begitu keras sehingga tidak pernah memakai atau memecahkan kepingan"". Sifat-sifat ""primer"" jirim telah dipinda kepada keterangan matematik, tidak seperti sifat-sifat ""sekunder"" seperti warna atau rasa. Seperti Descartes, Newton menolak sifat penting sifat sekunder. </s>",Bilakah Descartes dilahirkan? <sep> Apa yang ditulis oleh Descartes? <sep> Apa yang ditolak oleh Newton yang Descartes tidak? <sep> Apa yang dikatakan Descartes adalah sifat-sifat universal jirim? <sep> Kedua-dua sifat primer dan sekunder sesuai dengan bentuk keterangan apa? <sep>,e2e_qg
37927,"generate questions: Terdapat keseluruhan sastera mengenai ""struktur jirim"", bermula dari ""struktur elektrik"" pada awal abad ke-20, hingga ""struktur kuark jirim"" yang lebih baru, diperkenalkan hari ini dengan kenyataan: Memahami struktur kuark jirim telah menjadi salah satu kemajuan yang paling penting dalam fizik kontemporari. [penjelasan lanjut diperlukan] Dalam hubungan ini, ahli fizik bercakap tentang medan jirim, dan bercakap tentang zarah-zarah sebagai ""pengujaan kuantum bagi mod medan jirim"". Dan di sini terdapat petikan dari de Sabbata dan Gasperini: ""Dengan perkataan ""jirim"" kita menandakan, dalam konteks ini, sumber interaksi, iaitu medan spinor (seperti kuark dan lepton), yang dipercayai komponen asas jirim, atau medan skalar, seperti zarah Higgs, yang digunakan untuk memperkenalkan jisim dalam teori tolok (dan bahawa, bagaimanapun, boleh terdiri daripada medan fermion yang lebih asas)"". [penjelasan lanjut diperlukan]. </s>",Bilakah de Sabbata dan Gasperini menulis? <sep> Teori apa yang muncul selepas struktur kuark jirim? <sep> Memahami struktur elektrik telah membawa kepada kemajuan penting dalam bidang apa? <sep> Siapa yang menyifatkan zarah sebagai pengujaan kuantum? <sep> Teori apa yang menggunakan medan spinor? <sep>,e2e_qg
37928,"generate questions: Pada akhir abad ke-19 dengan penemuan elektron, dan pada awal abad ke-20, dengan penemuan nukleus atom, dan kelahiran fizik zarah, jirim dilihat sebagai terdiri daripada elektron, proton dan neutron yang berinteraksi untuk membentuk atom. Hari ini, kita tahu bahawa walaupun proton dan neutron tidak boleh dipisahkan, ia boleh dibahagikan kepada kuark, manakala elektron adalah sebahagian daripada keluarga zarah yang dipanggil lepton. Kedua-dua kuark dan lepton adalah zarah asas, dan kini dilihat sebagai juzuk asas jirim. </s>",Bidang fizik apa yang bermula pada abad ke-19? <sep> Apa bentuk atom? <sep> Apakah kuark yang dibahagikan kepada? <sep> Leptons terdiri daripada apa? <sep> Kita sekarang tahu bahawa kuark dan lepton bukan apa? <sep>,e2e_qg
37929,"generate questions: Kuark-kuark dan lepton ini berinteraksi melalui empat daya asas: graviti, elektromagnetisme, interaksi lemah, dan interaksi kuat. Model Piawai fizik zarah pada masa ini merupakan penjelasan terbaik untuk semua fizik, tetapi walaupun berpuluh-puluh tahun usaha, graviti belum boleh dipertanggungjawabkan pada tahap kuantum; ia hanya diterangkan oleh fizik klasik (lihat graviti kuantum dan graviti). Interaksi antara kuark dan lepton adalah hasil pertukaran zarah yang membawa daya (seperti foton) antara kuark dan lepton. Zarah-zarah yang membawa daya tidak sendiri membina blok. Sebagai satu akibat, jisim dan tenaga (yang tidak boleh dicipta atau dimusnahkan) tidak boleh sentiasa berkaitan dengan jirim (yang boleh dicipta daripada zarah bukan jirim seperti foton, atau bahkan keluar daripada tenaga tulen, seperti tenaga kinetik). Pembawa pasukan biasanya tidak

In [6]:
dev.tail(2)

,source_text,target_text,task
3269,"generate questions: Hubungan antara pasukan bukan konservatif makroskopik dan pasukan konservatif mikroskopik digambarkan oleh rawatan terperinci dengan mekanik statistik. Dalam sistem tertutup makroskopik, kuasa-kuasa bukan konservatif bertindak untuk mengubah tenaga dalaman sistem, dan sering dikaitkan dengan pemindahan haba. Menurut hukum Kedua termodinamik, kuasa-kuasa bukan konservatif semestinya menghasilkan transformasi tenaga dalam sistem tertutup daripada diperintahkan kepada keadaan-keadaan yang lebih rawak apabila entropi meningkat. </s>","Dalam rawatan apa yang digambarkan oleh pasukan bukan konservatif dan konservatif? <sep> Apa yang mengubah tenaga sistem tertutup makroskopik? <sep> Apakah pertukaran haba yang berkaitan? <sep> Apakah hukum termodinamik yang dikaitkan dengan pertukaran haba sistem tertutup? <sep> Apa yang membuat perubahan tenaga dalam sistem tertutup? <sep> Dalam sistem terbuka makroskopik, pasukan bukan konservatif bertindak untuk melakukan apa? <sep> Apa yang sering dikaitkan dengan pemindahan selesema? <sep> Pasukan konservatif sering dikaitkan dengan pemindahan apa? <sep> Apa yang tidak mengubah sistem tertutup makroskopik? <sep>",e2e_qg
3270,"generate questions: Kekuatan paun mempunyai rakan sejawat metrik, kurang biasa digunakan berbanding newton: kilogram-daya (kgf) (kadang kala kilopond), ialah daya yang dikenakan oleh graviti piawai pada satu kilogram jisim. The kilogram-force membawa kepada unit jisim alternatif, tetapi jarang digunakan: slug metrik (kadang-kadang mug atau hyl) adalah jisim yang memecut pada 1 m * s-2 apabila dikenakan kekuatan 1 kgf. The kilogram-force is not a part of the modern SI system, and is generally deprecated; however it still sees use for some purposes as expressing aircraft weight, jet thrust, bicycle spoke tension, torque wrench settings and engine output torque. Unit-unit arcane lain daya termasuk sthene, yang bersamaan dengan 1000 N, dan kip, yang bersamaan dengan 1000 lbf. </s>",Apakah istilah metrik yang kurang digunakan daripada Newton? <sep> Apakah kekuatan kilogram yang kadang-kadang dibalas sebagai? <sep> Apakah unit jisim yang sangat jarang digunakan dalam sistem metrik? <sep> Apakah istilah yang jarang digunakan unit daya sama dengan 1000 paun s daya? <sep> Apakah unit kuasa yang jarang digunakan bersamaan dengan seribu newton? <sep> Apa yang tidak mempunyai rakan metrik? <sep> Apakah kekuatan yang dikenakan oleh graviti standard pada satu tan jisim? <sep> Kekuatan apa yang membawa kepada unit jisim yang biasa digunakan? <sep> Kekuatan apa yang menjadi sebahagian daripada sistem SI moden? <sep>,e2e_qg


### Generate question based on text

In [7]:
from questiongenerator import QuestionGenerator
model = malaya.qa.extractive.huggingface(model = 'mesolitica/finetune-extractive-qa-flan-t5-base')

text_banjir = """ 
Banjir Malaysia 2021–2022 merupakan kejadian banjir 
yang melanda kawasan Semenanjung Malaysia pada Disember 2021. 
Banjir ini berlaku susulan hujan lebat berterusan di kebanyakan 
kawasan Semenanjung Malaysia berikutan lalunya sebuah lekukan tropika 
(nama rasmi Lekukan Tropika 29W) yang merentasi semenanjung dari Laut Cina Selatan. 
Di samping musim monsun serta kejadian air pasang, banjir mula terbit di beberapa negeri, 
secara ketaranya di kawasan Lembah Klang dan Pantai Timur."""


qg = QuestionGenerator()
banjir = qg.generate(text_banjir)

banjir

<pad> Apa yang melanda Semenanjung Malaysia pada Disember 2021? <sep> Apa yang berlaku di Semenanjung Malaysia pada Disember 2021? <sep> Apa nama lain untuk Lekukan Tropika 29W? <sep> Apa yang berlaku di Semenanjung Malaysia pada Disember 2021? <sep> Apa yang berlaku di Semenanjung Malaysia pada Disember 2021? <sep> Apa yang berlaku di Semenanjung Malaysia pada Disember 2021? <sep> Apa yang berlaku di Semenanjung Malaysia pada Disember 2021? <sep> Apa yang berlaku di Semenanjung Malaysia pada Disember 2021? <sep> Apa yang berlaku di Semenanjung Malaysia pada Disember 2021? <sep> Apa yang berlaku di Semenanjung Malaysia pada Disember 2021? <sep> Apa yang berlaku di Semenanjung Malaysia pada Disember 2021? <sep> </s>


['Apa yang melanda Semenanjung Malaysia pada Disember 2021?',
 'Apa yang berlaku di Semenanjung Malaysia pada Disember 2021?',
 'Apa nama lain untuk Lekukan Tropika 29W?']

### Answer generated question using malaya QA model

In [8]:
model.predict(text_banjir,banjir)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['Banjir Malaysia 2021–2022',
 'Banjir ini berlaku susulan hujan lebat ',
 'lekukan tropika']

In [9]:
from questiongenerator import QuestionGenerator



text = """Datu Paduka Mohammad Salleh bin Datu Balu, 
lebih dikenali dengan mudah sebagai Mat Salleh,
merupakan pejuang Sabah yang terkenal dalam penentangan 
terhadap penjajahan British di Sabah. 
Datu Paduka Mohammad Salleh bin Datu Balu dilahirkan di Inanam, 
Sabah dan berketurunan Suluk. Beliau memang terkenal dengan sifat 
kepahlawanan dan keberanian berperang. Beliau dikatakan pernah 
dilantik menjadi Wakil Kesultanan Sulu di Sungai Sugut dan daerah 
Tambunan ialah sebahagian daripada daerah di bawah kekuasaannya. 
Sebelum kedatangan British, beliau ialah seorang pemimpin tempatan yang berpengaruh."""

qg = QuestionGenerator()
question = qg.generate(text)

question

<pad> Apakah nama lain untuk Datu Paduka Mohammad Salleh bin Balu? <sep> Di manakah Datu Paduka Mohammad Salleh bin Balu dilahirkan? <sep> Di manakah Datu Paduka Mohammad Salleh bin Balu dilahirkan? <sep> Di manakah Datu Paduka Mohammad Salleh bin Balu dilahirkan? <sep> Di manakah Datu Paduka Mohammad Salleh bin Balu dilahirkan? <sep> </s>


['Apakah nama lain untuk Datu Paduka Mohammad Salleh bin Balu?',
 'Di manakah Datu Paduka Mohammad Salleh bin Balu dilahirkan?']

In [10]:

model.predict(text, question)

['Mat Salleh', 'Inanam, Sabah dan berketurunan Suluk']

**test question generation using sukanz dataset**

In [11]:
test.loc[20,'content']

['Malaysian Football League (MFL) mengumumkan hanya 10 pasukan yang bakal bertanding dalam divisyen kedua bola sepak Malaysia, Liga Premier 2022.',
 'Perkara itu disahkan oleh Ketua Pegawai Eksekutif (CEO) MFL, Stuart Ramalingam dalam satu kenyataan yang dikeluarkan pada hari ini.\xa0',
 'Dimaklumkan keputusan itu dibuat berikutan reserve team (pasukan simpanan), Perak FC II tersingkir secara automatik ke saingan Liga M3 selepas parent club (pasukan induk), Perak FC turun pangkat bermain di Liga Premier musim 2022.',
 'Kata Stuart, Perak FC menamatkan kempen di posisi ke-11 Liga Super 2021 sekaligus tersingkir ke Liga Premier 2022.\xa0',
 'Manakala pasukan berstatus reserve team secara automatik akan tersingkir daripada pertandingan sekiranya parent club mereka jatuh liga berdasarkan Manual Liga Malaysia Artikel 3.19.6 Peraturan Reserve Team.',
 'Jelasnya lagi, Amateur Football League (AFL) juga telah membuat pengumuman pembatalan Liga M3 pada 14 September 2021 menyaksikan tiada pasuka

In [12]:
test_sukan = ' '.join(test.loc[20,'content'])

question = qg.generate(test_sukan)

question

<pad> Berapa banyak pasukan yang akan bertanding dalam divisyen kedua bola sepak Malaysia? <sep> Siapa Ketua Pegawai Eksekutif MFL? <sep> Siapa Ketua Pegawai Eksekutif MFL? <sep> Siapa Ketua Pegawai Eksekutif MFL? <sep> Siapa Ketua Pegawai Eksekutif MFL? <sep> Berapa banyak pasukan yang akan bertanding dalam divisyen kedua bola sepak Malaysia? <sep> Berapa banyak pasukan yang akan bertanding dalam divisyen kedua bola sepak Malaysia? <sep> Siapa Ketua Pegawai Eksekutif MFL? <sep> Siapa Ketua Pegawai Eksekutif MFL? <sep> Berapa banyak pasukan yang akan bertanding dalam divisyen kedua bola sepak Malaysia? <sep> Berapa banyak pasukan yang akan bertanding dalam divisyen kedua bola sepak Malaysia?


['Berapa banyak pasukan yang akan bertanding dalam divisyen kedua bola sepak Malaysia?',
 'Siapa Ketua Pegawai Eksekutif MFL?']

In [13]:
model.predict(test_sukan, question)

Token indices sequence length is longer than the specified maximum sequence length for this model (708 > 512). Running this sequence through the model will result in indexing errors


['10', 'Stuart Ramalingam']